# Neurodesign comparison of design generators

In this notebook, we will compare 3 methods to generate an experimental design:
- a design optimised using the genetic algorithm
- a design optimised using simulations
- a randomly drawn design

We will do so using simulations: what is the resulting observed power when we simulate experiments according to the three designs.

## Requirements

Seaborn needs to be installed to plot some figures.

In [9]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import t

from neurodesign import Experiment, Optimisation

# for testing
cycles = 2 
sims = 2

output_dir = Path("output")
output_dir.mkdir(parents=True, exist_ok=True)

# cycles = 1000
# sims = 5000

## Optimise designs

First we define the experiment.
We will optimise an experiment with a TR of 2 seconds and 250 trials of 0.5 seconds each.
There are 4 stimulus types, and we are interested in the shared effect of the first and second stimulus versus baseline, as well as the difference between the first and the fourth stimulus.  We assume an autoregressive temporal autocorrelation of 0.3.

We sample ITI's from a truncated exponential distribution with minimum 0.3 seconds and maximum 4 seconds, and the mean is 1 second.

In [3]:
# define the experiment
exp = Experiment(
    TR=2,
    n_trials=450,
    P=[0.25, 0.25, 0.25],
    C=[[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, -1]],
    n_stimuli=3,
    rho=0.3,
    resolution=0.1,
    stim_duration=1,
    ITImodel="exponential",
    ITImin=0.3,
    ITImean=1,
    ITImax=4,
)

/home/remi/github/neurodesign/neurodesign/classes.py:461: UserWarning: the resolution is adjusted to be a multiple of the TR.New resolution: 0.1
  warnings.warn(
/home/remi/github/neurodesign/neurodesign/classes.py:620: RuntimeWarning: divide by zero encountered in log
  res = (h - 1) * np.log(s) + h * np.log(l) - l * s - np.log(gamma(h))


In [4]:
population_max = Optimisation(
    experiment=exp,
    weights=[0, 0.5, 0.25, 0.25],
    preruncycles=cycles,
    cycles=2,
    optimisation="GA",
)

population_max.optimise()

100% |##################################################################################################################################|
100% |##################################################################################################################################|


In [5]:
exp.FeMax = population_max.exp.FeMax
exp.FdMax = population_max.exp.FdMax

Below we define two populations of designs.  We will optimise one using the genetic algorithm, and the other using randomly drawn designs.  

We optimise for statistical power (weights = [0,1,0,0]).  We run 100 cycles.

In [10]:
population_ga = Optimisation(
    experiment=exp,
    weights=[0, 0.5, 0.25, 0.25],
    preruncycles=2,
    cycles=cycles,
    seed=1,
    outdes=5,
    I=10,
    folder=output_dir,
    optimisation="GA",
)

population_rn = Optimisation(
    experiment=exp,
    weights=[0, 0.5, 0.25, 0.25],
    preruncycles=2,
    cycles=cycles,
    seed=100,
    outdes=5,
    I=50,
    G=10,
    folder=output_dir,
    optimisation="simulation",
)

In [11]:
population_ga.optimise()

100% |##################################################################################################################################|


In [12]:
population_rn.optimise()

100% |##################################################################################################################################|


Below, we show how the efficiency scores improve over cycles for both algorithms, although the Genetic Algorithm clearly improves faster and reaches a higher plateau.

In [13]:
plt.plot(population_ga.optima, label="Genetic Algorithm")
plt.plot(population_rn.optima, label="Simulation")
plt.legend()
plt.show()
plt.savefig(output_dir / "test_scores.pdf")

/tmp/ipykernel_125437/2102259302.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Below, we repeat the random design generator, but we search only 100 designs and one generation.  As such, this is a random design.

In [14]:
# 1 gen
population_jo = Optimisation(
    experiment=exp,
    weights=[0, 0.5, 0.25, 0.25],
    preruncycles=1,
    cycles=1,
    seed=1,
    outdes=5,
    G=100,
    folder=output_dir,
    optimisation="simulation",
)
population_jo.optimise()

100% |##################################################################################################################################|


In [15]:
# collect scores and take average
scores = [x.F for x in population_jo.designs]

median_idx = np.where(scores == np.median(scores))[0][0]
rnd_median = population_jo.designs[median_idx]

# get PI
BTI_l = np.percentile(scores, 5)
BTI_u = np.percentile(scores, 95)

In [16]:
print(
    f"Optimisation score - random: {population_rn.optima[::-1][0]}\n"
    f"Optimisation score - genetic algorithm: {population_ga.optima[::-1][0]}\n"
    f"Optimisation score - simulation (90 percent PI): {BTI_l}-{BTI_u}"
)

Optimisation score - random: 0.8816239610114612
Optimisation score - genetic algorithm: 0.8786085367613324
Optimisation score - simulation (90 percent PI): 0.6902549450819885-0.8277075038744323


Let's look at the resulting experimental designs.

In [17]:
des = np.array([population_ga.bestdesign.Xconv, population_rn.bestdesign.Xconv, rnd_median.Xconv])
labels = ["Genetic Algorithm", "Simulation", "Median random design"]
plt.figure(figsize=(10, 7))
for ind, label in enumerate(labels):
    plt.subplot(3, 1, ind + 1)
    plt.plot(des[ind, :, :])
    plt.title(label)
    plt.tick_params(axis="x", which="both", bottom="off", labelbottom="off")

plt.savefig(output_dir / "designs.pdf")

In [18]:
des = np.array(
    [population_ga.bestdesign.Xconv, population_rn.bestdesign.Xconv] + [x.Xconv for x in population_jo.designs]
)

## Simulate data

We continue with the best designs from the two algorithms and the random design.  Below, we simulate data in one voxel that is significantly related to the task.   We assume beta values of (0.5, 0, -0.5).

In [19]:
# create datatables
tp = des.shape[1]
Y = np.zeros([tp, sims, des.shape[0]])

for i in range(sims):
    rnd = np.random.normal(0, 1, tp)
    for lb in range(Y.shape[2]):
        Y[:, i, lb] = np.dot(des[lb, :, :], np.array([0.5, 0, -0.5])) + rnd

ids = [0, 1, median_idx]

In [20]:
plt.plot(Y[:, 1:3, 1])

We analyse the data using `R` below.

In [21]:
%%R -i des,Y,sims,ids -o tvals_main,tvals_diff,pows
tvals_main <- array(NA,dim=c(sims,3))
tvals_diff <- array(NA,dim=c(sims,3))
pows <- array(NA,dim=c(dim(Y)[3],2))

threshold <- qt(0.95,df=(dim(des)[2]-2))

i = 1
for (method in 1:dim(Y)[3]){
    ts_main <- c()
    ts_diff <- c()
    for (sim in 1:sims){
        dif <- des[method,,1]-des[method,,2]
        fit_main <- lm(Y[,sim,method]~des[method,,])
        fit_diff <- lm(Y[,sim,method]~dif)
        ts_main[sim] <- summary(fit_main)$coef[2,3]
        ts_diff[sim] <- summary(fit_diff)$coef[2,3]
        }
    if ((method-1) %in% ids){
        tvals_main[,i] <- ts_main
        tvals_diff[,i] <- ts_diff
        i <- i+1
    }
    pows[method,1] <- mean(ts_main>threshold)
    pows[method,2] <- mean(ts_diff>threshold)
}

UsageError: Cell magic `%%R` not found.


This is what the distributions for the two contrasts look like.

In [22]:
from pathlib import Path

nms = ["Main effect", "Contrast effect"]
plt.figure(figsize=(18, 4))
for idx, tv in enumerate([tvals_main, tvals_diff]):
    plt.subplot(1, 2, idx + 1)
    for idy, method in enumerate(labels):
        sns.distplot(tv[:, idy], label=method)
    plt.title(nms[idx])
plt.legend()

plt.savefig(output_dir / "distributions.pdf")

NameError: name 'tvals_main' is not defined

In [23]:
pows.shape

NameError: name 'pows' is not defined

## Observed power

In [24]:
# We're assuming a single threshold on a single test, a representative simplification.
threshold = t.ppf(0.95, des.shape[1] - 2)
nms = ["main effect", "contrast effect"]
out = {label: [] for label in labels}
for idx in range(2):
    for idy, method in enumerate(labels):
        if idy < 2:
            print("The power for the %s with %s: %f" % (nms[idx], method, pows[idy, idx]))
    med = np.percentile(pows[2:, idx], 50)
    ll = np.percentile(pows[2:, idx], 5)
    ul = np.percentile(pows[2:, idx], 95)
    print("The median for the %s with a randomly drawn design: %f" % (nms[idx], med))
    print(
        "The 90 percent PI for the %s with a randomly drawn design: %f-%f"
        % (nms[idx], ll, ul)
    )

NameError: name 'pows' is not defined